#Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.1 MB/s eta 0:00:00


In [3]:

pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 3.7 MB/s eta 0:00:00


In [4]:
# Imports
import optuna
import plotly
import numpy as np
import polars as pl
import statistics as stats
from sklearn import model_selection
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import r2_score
import random

#Read in the dataset

In [5]:
# Data
df_train_x = pl.read_csv('./drive/MyDrive/'+'6002——draft/model/xtrain.csv')
df_test_x = pl.read_csv('./drive/MyDrive/'+'6002——draft/model/xvalid.csv')

df_train_y = pl.read_csv('./drive/MyDrive/'+'6002——draft/model/ytrain.csv')
df_test_y = pl.read_csv('./drive/MyDrive/'+'6002——draft/model/yvalid.csv')


In [6]:
df_train = df_train_x.with_columns(df_train_y)

In [7]:
df_train.glimpse()

Rows: 1952
Columns: 29
$ age              <f64> 73.0, 61.0, 64.0, 80.0, 68.0, 61.0, 80.0, 63.0, 78.0, 66.0
$ sex              <f64> 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0
$ education        <f64> 4.0, 3.0, 1.0, 3.0, 2.0, 4.0, 3.0, 1.0, 5.0, 3.0
$ marital_status   <f64> 1.0, 5.0, 3.0, 1.0, 1.0, 3.0, 2.0, 1.0, 1.0, 1.0
$ household_income <f64> 12.0, 4.0, 2.0, 6.0, 3.0, 7.0, 5.0, 3.0, 12.0, 4.0
$ diabetes         <f64> 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0
$ phq              <f64> 7.0, 3.0, 0.0, 0.0, 15.0, 3.0, 1.0, 10.0, 2.0, 0.0
$ func_score       <f64> 27.0, 34.0, 26.0, 21.0, 33.0, 20.0, 26.0, 45.0, 25.0, 20.0
$ avg_slp_dur      <f64> 9.90208333333334, 8.47708333333333, 7.84166666666667, 9.18541666666666, 6.4125, 10.7690476190476, 9.85416666666667, 9.1, 8.76458333333333, 10.8708333333333
$ std_slp_dur      <f64> 3.170516926207, 3.03725462788406, 1.14392806966672, 1.69727693908109, 1.1695118368396, 5.0748146159484, 3.05903229329613, 2.92772506804755, 2.7887449173930

In [8]:
df_test = df_test_x.with_columns(df_test_y)

In [9]:
df_test.glimpse()

Rows: 489
Columns: 29
$ age              <f64> 64.0, 66.0, 69.0, 80.0, 64.0, 80.0, 63.0, 65.0, 66.0, 70.0
$ sex              <f64> 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0
$ education        <f64> 5.0, 5.0, 2.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0
$ marital_status   <f64> 5.0, 1.0, 3.0, 2.0, 1.0, 1.0, 3.0, 1.0, 1.0, 3.0
$ household_income <f64> 5.0, 12.0, 4.0, 3.0, 6.0, 5.0, 5.0, 6.0, 7.0, 3.0
$ diabetes         <f64> 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0
$ phq              <f64> 8.0, 4.0, 0.0, 9.0, 0.0, 1.0, 1.0, 4.0, 0.0, 6.0
$ func_score       <f64> 31.0, 26.0, 21.0, 40.0, 20.0, 22.0, 21.0, 20.0, 21.0, 25.0
$ avg_slp_dur      <f64> 9.20416666666667, 10.1238095238095, 9.41875, 8.82083333333333, 8.47291666666666, 7.88958333333333, 7.53333333333333, 8.00208333333333, 7.91666666666667, 6.86111111111111
$ std_slp_dur      <f64> 1.97644361545666, 1.30131191434364, 3.01996580318071, 1.49935832836097, 1.89556578436548, 0.956470757275834, 0.959124931862702, 2.20634793950518,

In [10]:
predictors = df_train.columns[:-3]
target = 'af_score'

#Optuna + Catboost

In [13]:
# Suppress Optuna log messages
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Wrap everything in an Optuna trial object
def objective_cat(trial):

    # Suggest hyperparameters
    params = {'loss_function': 'MAE',
              'verbose': False,
              'random_seed': 20230706,
              'learning_rate': trial.suggest_float("learning_rate", 1e-2,
                                                   0.3, log = False),
              'iterations': trial.suggest_int("iterations", 50, 1500),
              'depth': trial.suggest_int("depth", 2, 10),
              'random_strength': trial.suggest_int('random_strength', 0.5, 10)
              }

    # Initialize list to store mae metrics
    mae_list = []
    rsq_list = []
    mse_list = []

    # Initializing stuff
    nfolds=10

    # Get names of features and targets
    X_cols = predictors
    y_cols = target

    # Create folds column
    df = df_train.with_columns(pl.lit(-1).alias('fold'))

    kf = model_selection.KFold(n_splits=nfolds, shuffle=True, random_state=random.randint(0,10000))
    for fold, (train_idx, valid_idx) in enumerate(kf.split(X=df.select(X_cols))):
        df[valid_idx, 'fold'] = fold

    # Save each split for one round of CV
    for k in range(nfolds):

        # Split
        tfold = df.filter(pl.col('fold') != k).drop('fold')
        vfold = df.filter(pl.col('fold') == k).drop('fold')

        ytrain = tfold.select(y_cols).to_numpy()
        yvalid = vfold.select(y_cols).to_numpy()

        xtrain = tfold.select(X_cols).to_numpy()
        xvalid = vfold.select(X_cols).to_numpy()

        # Get data into the correct format for CatBoost
        tpool = Pool(xtrain, ytrain)
        vpool = Pool(xvalid, yvalid)

        cat_model = CatBoostRegressor(**params)
        cat_model.fit(tpool, eval_set=vpool)

        # Predict, append, and output
        preds_train = cat_model.predict(tpool)
        preds_valid = cat_model.predict(vpool)

        mae_list.append(mean_absolute_error(yvalid, preds_valid))
        rsq_list.append(r2_score(yvalid, preds_valid))
        mse_list.append(mean_squared_error(yvalid, preds_valid))

    print(f'------------------------------------------')
    print(f'Mean R^2 for the AF model3: {stats.mean(rsq_list)}, std: {stats.stdev(rsq_list)}')
    print(f'Mean MSE for the AF model3: {stats.mean(mse_list)}, std: {stats.stdev(mse_list)}')
    print(f'Mean MAE for the AF model3: {stats.mean(mae_list)}, std: {stats.stdev(mae_list)}')
    # Write to a data frame and save
    df_output = pl.DataFrame().with_columns(
        pl.Series(name='rsq_af_model3', values=rsq_list),
        pl.Series(name='mse_af_model3', values=mse_list),
        pl.Series(name='mae_af_model3', values=mae_list)
    )
    with open('./drive/MyDrive/'+'6002——draft/model/Results-Model3.csv',
              mode="ab") as f:
      df_output.write_csv(f)
    # Return average validation MAE
    return stats.mean(mae_list)

# Set up and run the Optuna study
study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=100)
print(f'------------------------------------------')
print(f'Mean validation MAE for the best params: {study_cat.best_value}')
print(f'Best params: {study_cat.best_params}')

------------------------------------------
Mean R^2 for the AF model3: 0.08903307098671495, std: 0.053272357438443975
Mean MSE for the AF model3: 27.17890858319005, std: 2.131045430484369
Mean MAE for the AF model3: 4.085834009854055, std: 0.2066570499202773
------------------------------------------
Mean R^2 for the AF model3: 0.15257101082996385, std: 0.039241399005975244
Mean MSE for the AF model3: 25.367177413972932, std: 2.6958340037634
Mean MAE for the AF model3: 3.951262761763208, std: 0.23494095233022866
------------------------------------------
Mean R^2 for the AF model3: 0.10224811843511145, std: 0.03615485907394734
Mean MSE for the AF model3: 26.747555198121923, std: 1.766017783043257
Mean MAE for the AF model3: 4.0469144872313745, std: 0.15976631492526053
------------------------------------------
Mean R^2 for the AF model3: 0.11124797875991758, std: 0.044181289348227876
Mean MSE for the AF model3: 26.507532508884527, std: 1.9977297109562275
Mean MAE for the AF model3: 4.0

#Final Model

In [18]:
# Pre-process
X_cols = predictors
y_cols = target
xtrain = df_train.select(X_cols).to_numpy()
ytrain = df_train.select(y_cols).to_numpy()

xtest = df_test.select(X_cols).to_numpy()
ytest = df_test.select(y_cols).to_numpy()

train_pool = Pool(xtrain, ytrain)
test_pool = Pool(xtest)

# Define final params and fit the model
cat_best_params = {'loss_function': 'MAE',
                   'verbose': False,
                   'random_seed': 20230706,
                   'learning_rate': 0.10114190747954308,
                   'iterations': 633,
                   'depth': 5,
                   'random_strength': 0}

final_model = CatBoostRegressor(**cat_best_params).fit(train_pool)

# Get prediction and submit
preds_test = final_model.predict(test_pool)



In [19]:

print(f'Mean R^2 for the AF model3: {r2_score(ytest, preds_test)}')
print(f'Mean MSE for the AF model3: {mean_squared_error(ytest, preds_test)}')
print(f'Mean MAE for the AF model3: {mean_absolute_error(ytest, preds_test)}')

Mean R^2 for the AF model3: 0.089700764161734
Mean MSE for the AF model3: 26.897204678981684
Mean MAE for the AF model3: 4.133627704059086


#Feature Importance

In [16]:
# Installs
!pip install lets-plot
#imports
import plotly
import plotly.figure_factory as ff

from lets_plot import *
from lets_plot.bistro.corr import *
from lets_plot.mapping import as_discrete

# So the plots look nice
LetsPlot.setup_html()
plotly.offline.init_notebook_mode(connected = True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 9.2 MB/s eta 0:00:00


In [17]:
# Creating feature importance df
fi = final_model.get_feature_importance()
features = predictors

df_fi = pl.DataFrame().with_columns(
    pl.Series(name = 'Feature Importance', values = fi),
    pl.Series(name = 'Feature', values = features)
).sort('Feature Importance')

# Plotting
plt1 = \
    ggplot(df_fi)+\
    geom_point(aes(x = 'Feature Importance', y = 'Feature'),
               color = '#FF6666', size = 5)+\
    theme_minimal()+\
    theme(plot_title = element_text(hjust = 0.5, face = 'bold'),
         axis_line_y = element_line(size = 1),
         axis_line_x = element_line(size = 1),
         panel_grid_major_x = element_blank(),
         panel_grid_minor_x = element_blank(),
         panel_grid_minor_y = element_blank(),
         )+\
    labs(y = "Feature", x = 'Importance', title = 'CatBoost Feature Importance')

bunch = GGBunch()
bunch.add_plot(plt1, 0, 0, 700, 600)
bunch

